# **1: Créer les keywords à partir d'une phrase en se basant sur les mots d'un dictionnaire et un corpus de texte en passant par la tokenization, la correction, la lemmatization et le removeStopWords**
---
*importance: yes run it*
---
`fonction: SENTENCE_TO_CORRECT_WORDS`
---


In [1]:
import re
from collections import Counter
import unicodedata, re, string
import json 


def get_dico():
    textdir = "liste.de.mots.francais.frgut_.txt"
    try:DICO = open(textdir,'r',encoding="utf-8").read()
    except: DICO = open(textdir,'r').read()
    
    textdir = 'corpus_.txt'
    try:CORPUS = open(textdir,'r',encoding="utf-8").read();found=True
    except:pass
    try: CORPUS = open(textdir,'r').read();found=True 
    except: pass
    CORPUS = open(textdir,'r',encoding='cp1252').read();found=True 

    
    
    #WORDS = Counter(words( 'manger bouger difference update All edits that are one edit away from `word`. The subset of `words` that appear in the dictionary of WORDS '))
    
    return DICO+CORPUS


def remove_accents(input_str):
    '''
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii
    '''
    """This method removes all diacritic marks from the given string"""
    norm_txt = unicodedata.normalize('NFD', input_str)
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c))
    return unicodedata.normalize('NFC', shaved)

def clean_sentence(texte):
    # Replace diacritics
    texte = remove_accents(texte)
    # Lowercase the document
    texte = texte.lower()
    # Remove Mentions
    texte = re.sub(r'@\w+', '', texte)
    # Remove punctuations
    texte = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', texte)
    # Remove the doubled space
    texte = re.sub(r'\s{2,}', ' ', texte)
    #remove whitespaces at the beginning and the end
    texte = texte.strip()
    
    return texte

'''
#pas cool car il ya des nombres importants
def tokenize_sentence3(texte):
    #retourner les groupes d'alphabets
    return re.findall(r'\w+', texte.lower())
'''
'''
#inutile ici car sa VA est qu'ik decoupe en phrases
def tokenize_sentence2(texte):
        #clean the sentence
    blob_object = TextBlob(texte)
        #tokenize
    liste_words = blob_object.words
        #return 
    return liste_words
'''
def tokenize_sentence(texte):
        #clean the sentence 
    texte = clean_sentence(texte)
        #tokenize 
    liste_words = texte.split()
        #return 
    return liste_words

def strip_apostrophe(liste_words):
    get_radical = lambda word: word.split('\'')[-1]
    return list(map(get_radical,liste_words))

def pre_process(sentence):
    #remove '_' from the sentence 
    sentence = sentence.replace('_','')
    
    #get words fro the sentence 
    liste_words = tokenize_sentence(sentence)
    #cut out 1 or 2 letters ones 
    liste_words = [elt for elt in liste_words if len(elt)>2]
    #prendre le radical après l'apostrophe
    liste_words = strip_apostrophe(liste_words)
    print('\nsentence to words : ',liste_words)
    return liste_words







def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)
    
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])




def DICO_ET_CORRECTEUR():
    "cette fonction retourne la liste des mots de dictionnaire"
    DICO = get_dico()
    WORDS = Counter(pre_process(DICO)) #Counter prends un str et retourne une sorte de liste enrichie
    "correction des mots "
    N = sum(WORDS.values())
    P = lambda word: WORDS[word] / N #"Probability of `word`."
    
    correction = lambda word: max(candidates(word), key=P) #"Most probable
    return WORDS,correction

WORDS,CORRECTION = DICO_ET_CORRECTEUR()

#//https://www.ranks.nl/stopwords/french
#STOPWORDS =['alors', 'au', 'aucun', 'aussi', 'autre', 'avant', 'avec', 'avoir', 'bon', 'car', 'ce', 'cela', 'ces', 'ceux', 'chaque', 'ci', 'comme', 'comment', 'dans', 'des', 'du', 'dedans', 'dehors', 'depuis', 'devrait', 'doit', 'donc', 'dos', 'début', 'elle', 'elles', 'en', 'encore', 'essai', 'est', 'et', 'eu', 'fait', 'faites', 'fois', 'font', 'hors', 'ici', 'il', 'ils', 'je\tjuste', 'la', 'le', 'les', 'leur', 'là', 'ma', 'maintenant', 'mais', 'mes', 'mien', 'moins', 'mon', 'mot', 'même', 'ni', 'nommés', 'notre', 'nous', 'ou', 'où', 'par', 'parce', 'pas', 'peut', 'peu', 'plupart', 'pour', 'pourquoi', 'quand', 'que', 'quel', 'quelle', 'quelles', 'quels', 'qui', 'sa', 'sans', 'ses', 'seulement', 'si', 'sien', 'son', 'sont', 'sous', 'soyez\tsujet', 'sur', 'ta', 'tandis', 'tellement', 'tels', 'tes', 'ton', 'tous', 'tout', 'trop', 'très', 'tu', 'voient', 'vont', 'votre', 'vous', 'vu', 'ça', 'étaient', 'état', 'étions', 'été', 'être']
STOPWORDS = ["a","à","â","abord","afin","ah","ai","aie","ainsi","allaient","allo","allô","allons","après","assez","attendu","au","aucun","aucune","aujourd","aujourd'hui","auquel","aura","auront","aussi","autre","autres","aux","auxquelles","auxquels","avaient","avais","avait","avant","avec","avoir","ayant","b","bah","beaucoup","bien","bigre","boum","bravo","brrr","c","ça","car","ce","ceci","cela","celle","celle-ci","celle-là","celles","celles-ci","celles-là","celui","celui-ci","celui-là","cent","cependant","certain","certaine","certaines","certains","certes","ces","cet","cette","ceux","ceux-ci","ceux-là","chacun","chaque","cher","chère","chères","chers","chez","chiche","chut","ci","cinq","cinquantaine","cinquante","cinquantième","cinquième","clac","clic","combien","comme","comment","compris","concernant","contre","couic","crac","d","da","dans","de","debout","dedans","dehors","delà","depuis","derrière","des","dès","désormais","desquelles","desquels","dessous","dessus","deux","deuxième","deuxièmement","devant","devers","devra","différent","différente","différentes","différents","dire","divers","diverse","diverses","dix","dix-huit","dixième","dix-neuf","dix-sept","doit","doivent","donc","dont","douze","douzième","dring","du","duquel","durant","e","effet","eh","elle","elle-même","elles","elles-mêmes","en","encore","entre","envers","environ","es","ès","est","et","etant","étaient","étais","était","étant","etc","été","etre","être","eu","euh","eux","eux-mêmes","excepté","f","façon","fais","faisaient","faisant","fait","feront","fi","flac","floc","font","g","gens","h","ha","hé","hein","hélas","hem","hep","hi","ho","holà","hop","hormis","hors","hou","houp","hue","hui","huit","huitième","hum","hurrah","i","il","ils","importe","j","je","jusqu","jusque","k","l","la","là","laquelle","las","le","lequel","les","lès","lesquelles","lesquels","leur","leurs","longtemps","lorsque","lui","lui-même","m","ma","maint","mais","malgré","me","même","mêmes","merci","mes","mien","mienne","miennes","miens","mille","mince","moi","moi-même","moins","mon","moyennant","n","na","ne","néanmoins","neuf","neuvième","ni","nombreuses","nombreux","non","nos","notre","nôtre","nôtres","nous","nous-mêmes","nul","o","o|","ô","oh","ohé","olé","ollé","on","ont","onze","onzième","ore","ou","où","ouf","ouias","oust","ouste","outre","p","paf","pan","par","parmi","partant","particulier","particulière","particulièrement","pas","passé","pendant","personne","peu","peut","peuvent","peux","pff","pfft","pfut","pif","plein","plouf","plus","plusieurs","plutôt","pouah","pour","pourquoi","premier","première","premièrement","près","proche","psitt","puisque","q","qu","quand","quant","quanta","quant-à-soi","quarante","quatorze","quatre","quatre-vingt","quatrième","quatrièmement","que","quel","quelconque","quelle","quelles","quelque","quelques","quelqu'un","quels","qui","quiconque","quinze","quoi","quoique","r","revoici","revoilà","rien","s","sa","sacrebleu","sans","sapristi","sauf","se","seize","selon","sept","septième","sera","seront","ses","si","sien","sienne","siennes","siens","sinon","six","sixième","soi","soi-même","soit","soixante","son","sont","sous","stop","suis","suivant","sur","surtout","t","ta","tac","tant","te","té","tel","telle","tellement","telles","tels","tenant","tes","tic","tien","tienne","tiennes","tiens","toc","toi","toi-même","ton","touchant","toujours","tous","tout","toute","toutes","treize","trente","très","trois","troisième","troisièmement","trop","tsoin","tsouin","tu","u","un","une","unes","uns","v","va","vais","vas","vé","vers","via","vif","vifs","vingt","vivat","vive","vives","vlan","voici","voilà","vont","vos","votre","vôtre","vôtres","vous","vous-mêmes","vu","w","x","y","z","zut","alors","aucuns","bon","devrait","dos","droite","début","essai","faites","fois","force","haut","ici","juste","maintenant","mine","mot","nommés","nouveaux","parce","parole","personnes","pièce","plupart","seulement","soyez","sujet","tandis","valeur","voie","voient","état","étions"]
STOPWORDS = list(map(remove_accents,STOPWORDS))




with open("sample_.json",'r',encoding='cp1252') as json_file:
    #json_file.seek(0)
    LISTE = json.load(json_file)
my_stemmer = lambda word: LISTE[word] if word in LISTE else word

def SENTENCE_TO_CORRECT_WORDS(sentence):
    "cette fonction retourne la liste des mots du user"
    print('\n------------pre_process--------\n')
    liste_words = pre_process(sentence)
    print(liste_words)
    print('\n------------correction--------\n')
    liste_words = list(map(CORRECTION,liste_words))
    print(liste_words)
    print('\n------------stemming--------\n')
    liste_words = list(map(my_stemmer,liste_words))
    print(liste_words)
    print('\n------------remove stop-words--------\n')
    liste_words = [elt for elt in liste_words if elt not in STOPWORDS]
    print(liste_words)
    print('\n-------------------------------------\n')
    return liste_words


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# **test**
---
*importance: canTest*
---

In [2]:

out = 0 

if __name__ == '__main__':
    print('\n-------------------------------------\n')
    sentence = 'voilà ma phrase'
    print('sentence: ',sentence)
    liste_words = SENTENCE_TO_CORRECT_WORDS(sentence)
    print('liste_words:',liste_words)
    print('\n-------------------------------------\n')
    print('\ndes phrases à mots raté, d\'une faute ou deux, à corriger')
    while out!=2:
        sentence = input('sentence or word: ')
        
        if sentence: 
            #CORRECTION(word.lower())
            liste_words = SENTENCE_TO_CORRECT_WORDS(sentence)
            #print(liste_words)
        else: out +=1



-------------------------------------

sentence:  voilà ma phrase

------------pre_process--------


sentence to words :  ['voila', 'phrase']
['voila', 'phrase']

------------correction--------

['voila', 'phrase']

------------stemming--------

['voiler', 'phrase']

------------remove stop-words--------

['voiler', 'phrase']

-------------------------------------

liste_words: ['voiler', 'phrase']

-------------------------------------


des phrases à mots raté, d'une faute ou deux, à corriger
sentence or word: 
sentence or word: 


In [3]:
SENTENCE_TO_CORRECT_WORDS('messaeges d\'erreues. Le founiseur ARIBA n\'existeaint pas')


------------pre_process--------


sentence to words :  ['messaeges', 'erreues', 'founiseur', 'ariba', 'existeaint', 'pas']
['messaeges', 'erreues', 'founiseur', 'ariba', 'existeaint', 'pas']

------------correction--------

['messages', 'erreurs', 'founisseur', 'ariba', 'existaient', 'pas']

------------stemming--------

['message', 'erreur', 'founisseur', 'ariba', 'exister', 'pas']

------------remove stop-words--------

['message', 'erreur', 'founisseur', 'ariba', 'exister']

-------------------------------------



['message', 'erreur', 'founisseur', 'ariba', 'exister']

# **1-2: un exemple de lemmatizer**
---
*importance: run if `cosine_similarity_T(mine=False)`*
---

In [4]:
import nltk
'''
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
'''

"\nnltk.download('punkt')\nnltk.download('wordnet')\nnltk.download('averaged_perceptron_tagger')\nnltk.download('stopwords')\n"

In [5]:
import re
import pandas as pd
import operator
import nltk 
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
def wordLemmatizer(data,colname):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('french') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,colname] = str(Final_words)
                file_clean_k.loc[index,colname] = str(Final_words)
                file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)

    return file_clean_k


def wordLemmatizer_(sentence):
    #prendre une phrase que retourner un str (les mots sont separes par des ,)
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", sentence).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    idx = 0
    colname = 'keyword_final'
    q_df.loc[idx,'q_clean'] =tokens
    print('\n\n---inputtoken');print(q_df.q_clean)
    print('\n\n---outputlemma');print(wordLemmatizer(q_df.q_clean,colname).loc[idx,colname])
    return wordLemmatizer(q_df.q_clean,colname).loc[idx,colname]

# **2: trouver la meilleure phrase dans une liste de phrase**
---
*importance: yes run it*
---
`fonction: cosine_similarity_T`
---

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
import numpy as np 
import time


def init(df_news):
  print('init!!!')
  ## Create Vocabulary
  vocabulary = set()
  for doc in df_news.Clean_Keyword:
      vocabulary.update(doc.split(','))
  vocabulary = list(vocabulary)# Intializating the tfIdf model
  tfidf = TfidfVectorizer(vocabulary=vocabulary)# Fit the TfIdf model
  tfidf.fit(df_news.Clean_Keyword)# Transform the TfIdf model
  tfidf_tran=tfidf.transform(df_news.Clean_Keyword)
  globals()['vocabulary'],globals()['tfidf'],globals()['tfidf_tran'] = vocabulary,tfidf,tfidf_tran

def gen_vector_T(tokens,df_news,vocabulary,tfidf,tfidf_tran):
  Q = np.zeros((len(vocabulary)))    
  x= tfidf.transform(tokens)
  #print(tokens[0].split(','))
  #print(keywords)
  for token in tokens[0].split(','):
      
      try:
          ind = vocabulary.index(token)
          Q[ind]  = x[0, tfidf.vocabulary_[token]]
          print(token,':',ind)
      except:
          print(token,':','not found')
          pass
  return Q
def cosine_sim(a, b):
    if not np.linalg.norm(a) and not np.linalg.norm(b): return -3
    if not np.linalg.norm(a):return -1
    if not np.linalg.norm(b):return -2
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim   
def cosine_similarity_T(k, query,df_news,vocabulary=None,tfidf=None,tfidf_tran=None,mine=True):
    try:
      vocabulary = globals()['vocabulary']
      tfidf = globals()['tfidf']
      tfidf_tran = globals()['tfidf_tran']
    except:
      print('up exception')
      init(df_news)
    q_df = pd.DataFrame(columns=['q_clean'])
    if mine:q_df.loc[0,'q_clean'] =','.join(SENTENCE_TO_CORRECT_WORDS(query))
    else:q_df.loc[0,'q_clean'] = wordLemmatizer_(query)
    
    
    print('\n\n---q_df');print(q_df)
    
    print('\n\n')
    d_cosines = []
    query_vector = gen_vector_T(q_df['q_clean'],df_news,vocabulary,tfidf,tfidf_tran )
    for d in tfidf_tran.A:
        d_cosines.append(cosine_sim(query_vector, d ))
                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Subject'] = df_news['Subject'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a


def test(data,sentence,init_=False,mine=True):
  if not init_:
    deb = time.time();print('\n\n###########')
    init(df_news)
    print('\n###########temps init: ', time.time()-deb)
  deb = time.time();print('\n\n###########')
  print(cosine_similarity_T(10, sentence,df_news))
  print('\n###########temps methode 1: ', time.time()-deb)

# **Example1:**
---
`BDD: df_news`
---
*importance: nope*
---

In [7]:
import pandas as pd 
import re 
from nltk.tokenize import word_tokenize


news = pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/newsgroups.json')



for i,txt in enumerate(news['content']):
    subject = re.findall('Subject:(.*\n)',txt)
    if (len(subject) !=0):
        news.loc[i,'Subject'] =str(i)+' '+subject[0]
    else:
        news.loc[i,'Subject'] ='NA'
df_news =news[['Subject','content']]



df_news.content =df_news.content.replace(to_replace='from:(.*\n)',value='',regex=True) ##remove from to email 
df_news.content =df_news.content.replace(to_replace='lines:(.*\n)',value='',regex=True)
df_news.content =df_news.content.replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]',value=' ',regex=True) #remove punctuation except
df_news.content =df_news.content.replace(to_replace='-',value=' ',regex=True)
df_news.content =df_news.content.replace(to_replace='\s+',value=' ',regex=True)    #remove new line
df_news.content =df_news.content.replace(to_replace='  ',value='',regex=True)                #remove double white space
df_news.content =df_news.content.apply(lambda x:x.strip())  # Ltrim and Rtrim of whitespace



df_news['content']=[entry.lower() for entry in df_news['content']]
df_news['Word tokenize']= [word_tokenize(entry) for entry in df_news.content]





/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

# **Example1: plus rapide**
---
`BDD: df_news`
---
*importance: canTest*
---

In [8]:
import pandas as pd 
df_news = pd.read_json('https://raw.githubusercontent.com/zayedrais/DocumentSearchEngine/master/data/WordLemmatize20NewsGroup.json')
df_news.head()
df_news.info()
df_news.loc[0,'Clean_Keyword']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11314 entries, 0 to 11313
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Subject        11314 non-null  object
 1   Clean_Keyword  11314 non-null  object
dtypes: object(2)
memory usage: 265.2+ KB


'thing,car,nntp,post,host,university,maryland,college,park,line,wonder,anyone,could,enlighten,car,saw,day,sport,car,look,late,early,call,bricklin,door,really,small,addition,front,bumper,separate,rest,body,know,anyone,tellme,model,name,engine,spec,year,production,car,make,history,whatever,info,funky,look,car,please,mail,il,bring,neighborhood,lerxst'

# **Text example1**
---
`output: mesures de similarités`
---
*importance: canTest*
---

In [9]:
init

<function __main__.init>

In [12]:
sentence = "computer science"
test(df_news,sentence,init_=False,mine=True) #il v bugger sur l'anglais à moins que je remplace les data (stop_words et corpus et tout)
sentence = "state diagrams"
test(df_news,sentence,init_=True,mine=True) 



###########
init!!!

###########temps init:  3.1772680282592773


###########

------------pre_process--------


sentence to words :  ['computer', 'science']
['computer', 'science']

------------correction--------

['computer', 'science']

------------stemming--------

['computer', 'science']

------------remove stop-words--------

['computer', 'science']

-------------------------------------



---q_df
            q_clean
0  computer,science



computer : 39988
science : 7641
   index                                            Subject     Score
0   2231             2231 Computer Engr vs Computer Science  0.396137
1  10340                      10340 Science and Methodology  0.302603
2   4173                   4173 Rawlins debunks creationism  0.284936
3   4326             4326 Computer Engr vs Computer Science  0.284256
4   6921            6921 Automatic layout of state diagrams  0.260732
5   7618  7618 Solution Why do I need xrdb m when Xdefau...  0.251170
6   5741                 

# **Example2:**
---
`BDD: df_new`
---
*importance: yes: run it*
---

In [13]:
def open_file(textdir):
  found = False
  try:texte = open(textdir,'r',encoding="utf-8").read();found=True
  except:pass
  try: texte = open(textdir,'r').read();found=True 
  except: pass
  if not found:
    texte = open(textdir,'r',encoding='cp1252').read();found=True
  return  texte
def add_col(df_news,titre,keywords):
  return df_news.append(dict(zip(df_news.columns,[titre, keywords])), ignore_index=True)
SENTENCE_TO_CORRECT_WORDS('La PR reste au statut «\xa0Approuve(e)\xa0» et il n’y a pas de commande\"\'')


------------pre_process--------


sentence to words :  ['reste', 'statut', 'approuve', 'n’y', 'pas', 'commande']
['reste', 'statut', 'approuve', 'n’y', 'pas', 'commande']

------------correction--------

['reste', 'statut', 'approuve', 'n’y', 'pas', 'commande']

------------stemming--------

['rester', 'statut', 'approuver', 'n’y', 'pas', 'commander']

------------remove stop-words--------

['rester', 'statut', 'approuver', 'n’y', 'commander']

-------------------------------------



['rester', 'statut', 'approuver', 'n’y', 'commander']

In [14]:
liste_pb = [elt for elt in open_file('liste_pb_.txt').split('\n') if elt]
df_new = df_news.drop(df_news.index)
for i,titre in enumerate(liste_pb):
  keywords = ','.join(SENTENCE_TO_CORRECT_WORDS(titre))
  df_new = add_col(df_new,titre,keywords)
df_new.head()


------------pre_process--------


sentence to words :  ['message', 'erreur', 'fournisseur', 'ariba', 'existe', 'pas']
['message', 'erreur', 'fournisseur', 'ariba', 'existe', 'pas']

------------correction--------

['message', 'erreur', 'fournisseur', 'ariba', 'existe', 'pas']

------------stemming--------

['message', 'erreur', 'fournisseur', 'ariba', 'exister', 'pas']

------------remove stop-words--------

['message', 'erreur', 'fournisseur', 'ariba', 'exister']

-------------------------------------


------------pre_process--------


sentence to words :  ['message', 'erreur', 'commande', 'd’article', 'non', 'autorise', 'otp']
['message', 'erreur', 'commande', 'd’article', 'non', 'autorise', 'otp']

------------correction--------

['message', 'erreur', 'commande', 'd’article', 'non', 'autorise', 'otp']

------------stemming--------

['message', 'erreur', 'commander', 'd’article', 'non', 'autoriser', 'otp']

------------remove stop-words--------

['message', 'erreur', 'commander', '

,Subject,Clean_Keyword
0,"Message d'erreur : ""Le fournisseur ARIBA n'exi...","message,erreur,fournisseur,ariba,exister"
1,"Message d'erreur : ""Commande d’article non aut...","message,erreur,commander,d’article,autoriser,otp"
2,"Message d'erreur : ""Statut utilisateur FERM ac...","message,erreur,statut,utilisateur,ferm,actif,otp"
3,"Message d'erreur : ""Statut systeme TCLO actif ...","message,erreur,statut,systeme,tclo,actif,ord"
4,"Message d'erreur ""___ Cost center change could...","message,erreur,cost,center,changer,could,not,e..."


# **Test Example2**
---
`BDD: df_new`
---
*importance: yes run it*
---

In [15]:
sentence = 'Message d\'erreur \"La qte livree est differente de la qte facturee ; fonction impossible"'
sentence = 'erreur de conversion'
sentence = 'message d\'erreur'
sentence = "le fournisseur MDM n'existe pas"
sentence = "groupe d'acheteurs non défini"
sentence = "UO4"
init(df_new) 

cosine_similarity_T(10,sentence,df_new )

init!!!

------------pre_process--------


sentence to words :  ['uo4']
['uo4']

------------correction--------

['uo4']

------------stemming--------

['uo4']

------------remove stop-words--------

['uo4']

-------------------------------------



---q_df
  q_clean
0     uo4



uo4 : 155


,index,Subject,Score
0,65,UO4-2 Commande | Demande de support modificati...,0.428515
1,57,UO4-5 Commande | Envoi d'une commande manuelle,0.424080
2,30,"Message d'erreur ""Renseigner correctement le d...",0.000000
3,29,"Message d'erreur ""Article ___ non gere dans la...",0.000000
4,28,"Message d'erreur ""Fonctions oblig. Suivantes n...",0.000000
5,27,"Message d'erreur ""Qte (+ tolerance livr.excede...",0.000000
6,26,"Message d'erreur ""Erreur de conversion des qua...",0.000000
7,25,"Message d'erreur ""Le cpte general ___ ne peut ...",0.000000
8,16,"Message d'erreur ""Utilisateur ___ traite deja ...",0.000000
9,24,"Message d'erreur ""Le founisseur XXXX-40 n’exis...",0.000000


# **3:non-utilisé: Créer les keywords à partir d'une phrase en se basant sur les mots d'un dictionnaire en passant par la tokenization et le removeStopWords**
---
`BDD: df_news`
---
*importance: nope*
---

In [16]:
#https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb
import re
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
with open('stp_words_.txt','r') as f:
    Stop_words_strong = f.read()



Stop_words_strong = re.findall(r'\w+',Stop_words_strong)
Stop_words_strong.append("quelqu")
docs ='''Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment Trouvé apparemment *t sur le parapet de ce mur que s'ouvraient les ailes de la jeunesse. Douce et sainte illusion de l'espoir de la vaincre. Assez avisé pour juger tout effet inutile dans ce cabinet où l'air est si doux, vous ne faites jamais rien sans raison, que c'en était fait du mal. Accoutumé à vivre du présent, nous savons depuis deuxabinet où l'air est si doux, vous ne faites jamais rien sans raison, que c'en était fait du mal. Accoutumé à vivre du présent, nous savons depuis deuxabinet où l'air est si doux, vous ne faites jamais rien sans raison, que c'en était fait du mal. Accoutumé à vivre du présent, nous savons depuis deux jours. Comprenant tout à coup renaissante animait d'une sève nouvelle, sans qu'aucune des précédentes. Abandonnez votre jeu et asseyez-vous à côté de l'avenir de notre entreprise. Planter là, et il reconnut bientôt que c'était impossible de bouger ou de parler. Allez-vous-en, et soyez comme une petite désespérée vers la mule. parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet parapet'''

sentence = re.sub('[?,.]','',docs)
listes_phrases = sentence.split(' ')
cv = CountVectorizer(max_df=0.85,stop_words=Stop_words_strong,max_features=10000)


word_count_vector = cv.fit_transform(listes_phrases)

#Now, let's look at 10 words from our vocabulary. Sweet, these are mostly programming related.
liste = list(cv.vocabulary_.keys())[:]

#>>> liste
#['trouvé', 'apparemment', 'parapet', 'mur', 'ouvraient', 'ailes', 'jeunesse', 'douce', 'sainte', 'illusion', 'espoir', 'vaincre', 'avisé', 'juger', 'inutile', 'cabinet', 'air', 'doux', 'jamais', 'raison', 'mal', 'accoutumé', 'vivre', 'présent', 'savons', 'jours', 'comprenant', 'coup', 'renaissante', 'animait', 'sève','nouvelle', 'précédentes', 'abandonnez', 'jeu', 'asseyez', 'côté', 'avenir', 'entreprise', 'planter', 'reconnut', 'bientôt', 'impossible', 'bouger', 'parler', 'allez', 'petite', 'désespérée', 'mule']

#We can also get the vocabulary by using get_feature_names()
liste2 = list(cv.get_feature_names())[2000:2015]

# you only needs to do this once
feature_names=cv.get_feature_names()
print('keywords')
feature_names[:10]

keywords


['abandonnez',
 'accoutumé',
 'ailes',
 'air',
 'allez',
 'animait',
 'apparemment',
 'asseyez',
 'avenir',
 'avisé']